In [0]:
dbutils.widgets.text(
    "output_table",
    "tsfrt.gsa.document_base2",
    label="source for chunk content",
)

dbutils.widgets.text(
    "vector_index_table",
    "document_base",
    label="name of pgvector index",
)

In [0]:
from pyspark.sql.functions import col, lit, concat_ws, to_json

chunk_table = dbutils.widgets.get("output_table")

# Read the table
df = spark.read.table(chunk_table)

# Select specific columns and rename them
df = df.select(
    col("id"),
    col("doc_id").alias("doc_url"),
    col("transcription").alias("content"),
    to_json(col("embedding")).alias("embedding")
).withColumn("sensitivity", lit("public"))

df.printSchema()
display(df)

root
 |-- id: integer (nullable = true)
 |-- doc_url: string (nullable = true)
 |-- content: string (nullable = true)
 |-- embedding: string (nullable = true)
 |-- sensitivity: string (nullable = false)



id doc_url content embedding sensitivity 1 /Volumes/tsfrt/gsa/performance/FY 2024 GSA Annual Performance Plan FY 2022 Report_Final_508.pdf ERROR: Error code: 400 - {'error_code': 'BAD_REQUEST', 'message': 'Invalid base64 string for image\n'} [-0.54931640625,-0.54931640625,0.344482421875,-0.50732421875,0.3525390625,-0.1373291015625,-0.401123046875,1.0,0.5263671875,0.8388671875,0.3291015625,0.2435302734375,-0.289794921875,-0.329833984375,-0.8720703125,0.08245849609375,-0.372802734375,-0.986328125,-0.2183837890625,-0.2193603515625,-0.0775146484375,-0.306884765625,0.07501220703125,-0.90771484375,0.271728515625,0.66552734375,-0.87451171875,-0.6982421875,0.293701171875,0.50927734375,0.0011501312255859375,0.1204833984375,-1.982421875,-0.37451171875,0.480224609375,-0.2061767578125,-0.86865234375,1.28515625,0.79736328125,0.1727294921875,0.66015625,-0.301025390625,-1.1953125,-0.37841796875,-0.1119384765625,0.7490234375,-0.31689453125,0.51904296875,1.0302734375,0.50244140625,-0.056243896484375,0.50244140625,0.62939453125,0.1533203125,0.280517578125,0.583984375,0.2161865234375,0.420654296875,0.430419921875,0.430419921875,-0.201416015625,1.23828125,-0.1707763671875,0.13232421875,0.1295166015625,1.6875,0.6572265625,0.62841796875,-0.7314453125,0.498779296875,0.37890625,-1.0302734375,-0.37890625,0.451904296875,0.3037109375,0.358642578125,0.5244140625,0.06781005859375,0.37060546875,0.1583251953125,0.79541015625,0.03985595703125,0.6806640625,-0.424072265625,-0.90380859375,0.70751953125,1.0419921875,-0.2220458984375,-0.00762176513671875,-1.2197265625,-0.414306640625,-1.2265625,-0.054962158203125,-0.56640625,-0.517578125,-0.475830078125,0.7412109375,-0.7763671875,-1.0615234375,-0.180419921875,0.1251220703125,-2.064453125,0.76904296875,-0.30517578125,-0.50634765625,0.505859375,0.05694580078125,0.88818359375,0.4384765625,0.61767578125,1.1005859375,0.409912109375,0.98681640625,-0.98583984375,-0.68701171875,-0.245849609375,-0.053497314453125,-0.11138916015625,0.05755615234375,0.438720703125,-1.0283203125,-0.0958251953125,0.7548828125,0.49462890625,0.62109375,0.09332275390625,0.50048828125,0.23681640625,-0.8701171875,0.76171875,0.411865234375,0.92529296875,-0.37060546875,-0.0732421875,-0.36669921875,0.4921875,0.10858154296875,0.450927734375,0.66357421875,-0.53955078125,-0.63330078125,-1.5517578125,0.010528564453125,-0.2327880859375,0.311767578125,0.219970703125,0.04150390625,0.135986328125,-0.66162109375,-0.494873046875,0.58935546875,-0.88916015625,0.1466064453125,-0.1962890625,0.51513671875,-0.2275390625,0.14599609375,-2.2265625,0.5205078125,-1.3212890625,-1.5224609375,-0.685546875,-0.423583984375,0.395751953125,-0.09271240234375,-0.2030029296875,-1.0947265625,-1.1689453125,1.03125,0.7490234375,-0.158935546875,-0.328369140625,-1.0224609375,-0.6845703125,-0.5859375,-0.319091796875,0.2216796875,-0.24755859375,-0.395263671875,-0.07342529296875,-0.69775390625,0.1630859375,-0.038818359375,-1.2607421875,1.0224609375,0.2174072265625,-0.05810546875,0.0084075927734375,-0.85595703125,-0.426025390625,0.331298828125,-1.8056640625,-0.3017578125,0.28564453125,-0.6494140625,-0.8916015625,-0.2939453125,0.350341796875,-0.806640625,-0.11663818359375,0.64013671875,0.23974609375,-0.311767578125,-0.159912109375,-0.07684326171875,0.72265625,0.048370361328125,-0.241943359375,-0.24169921875,-0.7255859375,-0.48681640625,0.045867919921875,-0.471435546875,0.53369140625,0.03570556640625,0.806640625,-0.888671875,-0.265380859375,0.044647216796875,0.24267578125,0.31396484375,-0.3310546875,-0.52978515625,0.34814453125,0.75732421875,-1.3798828125,0.5986328125,0.197021484375,-0.282958984375,1.125,-0.1396484375,-0.78662109375,-0.25341796875,0.00614166259765625,0.403564453125,-0.77685546875,-0.48779296875,-0.33740234375,0.303955078125,-0.6875,0.86669921875,-0.533203125,-0.0184478759765625,0.69091796875,0.7431640625,0.08831787109375,1.294921875,-1.111328125,-1.087890625,-0.059539794921875,-0.1513671875,0.9765625,1.0107421875,-1.1884765625,-0.79345703125,-0.458251953125,-0.2724609375,-0.

In [0]:
import pandas as pd
import json
import psycopg2
from psycopg2.extras import execute_values

vector_index = dbutils.widgets.get("vector_index_table")

connection_config = dbutils.secrets.get("gsa-rag", "vector-conn-params")

pdf = df.toPandas()

for i in range(253):
    # Establish a connection to your PostgreSQL database
    data = pdf.iloc[i:i+1].copy()
    
    # Convert data types to Python native types
    data = data.astype(object).where(pd.notnull(data), None)
    
    conn = psycopg2.connect(**json.loads(connection_config))
    
    cursor = conn.cursor()
    # Prepare data for insertion (list of tuples)
    data_to_insert = [tuple(row) for row in data.itertuples(index=False, name=None)]
    # Define the SQL INSERT statement
    table_name = vector_index
    columns = ", ".join(data.columns)
    
    # Use ON CONFLICT DO UPDATE for merging (upsert)
    # Replace 'id' with your primary key or unique constraint
    insert_sql = f"""
    INSERT INTO {table_name} ({columns})
    VALUES %s
    ON CONFLICT (id) DO UPDATE SET
        content = EXCLUDED.content,
        doc_url = EXCLUDED.doc_url,
        sensitivity = EXCLUDED.sensitivity,
        embedding = EXCLUDED.embedding;
    """
    
    # Execute the bulk insertion/update
    try:
        execute_values(cursor, insert_sql, data_to_insert)
        conn.commit()
        print(f"{len(data_to_insert)} rows successfully merged into {table_name}.")
    except Exception as e:
        conn.rollback()
        print(f"{i}Error merging data: {e}")
    finally:
        cursor.close()
        conn.close()

1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows successfully merged into document_base.
1 rows succes